<a href="https://colab.research.google.com/github/belloncj/tone_financial_news/blob/main/EventStudies_Santana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
from sklearn import linear_model

import scipy.stats

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Import economic data
allPrices = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/exxon_stock.csv')
factors = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/3_factors.csv')
events = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/exxon_ft_title_outliers.csv')

In [ ]:
allPrices = allPrices.dropna()
factors = factors.dropna()
events = events.dropna()

In [ ]:
allPrices['Date'] = pd.to_datetime(allPrices['Date'])

# Set the "Date" column as the index
allPrices.set_index(allPrices['Date'].dt.strftime('%Y%m%d'), inplace=True)

# Remove the "Date" column
allPrices.drop(columns=['Date'], inplace=True)

# Remove the first column ("Unnamed: 0")
allPrices.drop(columns=['Unnamed: 0'], inplace=True)


In [ ]:
# Calcular el retorno porcentual
allPrices['Return'] = allPrices['Close'].pct_change()
allPrices = allPrices.dropna()

In [ ]:

allPrices

,Open,High,Low,Close,Adj Close,Volume,Return
Date,,,,,,,
20080103,93.860001,94.739998,93.510002,93.830002,53.735184,19160600,0.003422
20080104,93.239998,94.360001,91.879997,92.080002,52.732998,24730900,-0.018651
20080107,92.500000,93.150002,90.250000,91.220001,52.240498,28937500,-0.009340
20080108,91.860001,92.050003,89.680000,90.050003,51.570442,21764500,-0.012826
20080109,90.110001,91.730003,89.500000,91.559998,52.435207,26198800,0.016768
...,...,...,...,...,...,...,...
20171222,83.879997,84.019997,83.599998,83.970001,63.519855,10161400,0.001431
20171226,83.959999,84.360001,83.900002,83.980003,63.527409,4777200,0.000119
20171227,83.989998,84.099998,83.739998,83.900002,63.466904,7000600,-0.000953


In [ ]:
factors.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# separate RF
rf=factors[['Date', 'RF']]
rf=rf.set_index('Date')

# Set the "Date" column as the index
factors.set_index('Date', inplace=True)

factors.drop('RF',axis=1)
factors['Constant']=1
factors = factors[['Constant','Mkt-RF', 'SMB', 'HML']]

In [ ]:
factors = factors.loc['20080103':'20230201']

In [ ]:
factors

,Constant,Mkt-RF,SMB,HML
Date,,,,
20080103,1,-0.14,-1.01,-0.41
20080104,1,-2.56,-0.48,0.05
20080107,1,0.16,0.12,0.18
20080108,1,-1.81,-0.43,-0.94
20080109,1,1.10,-0.33,-0.66
...,...,...,...,...
20230126,1,1.08,-0.58,0.01
20230127,1,0.36,0.62,-1.16
20230130,1,-1.38,-0.10,0.72


In [ ]:
events['Date'] = pd.to_datetime(events['Date'])

# Set the "Date" column as the index
events.set_index(events['Date'].dt.strftime('%Y%m%d'), inplace=True)

# Remove the "Date" column
events.drop(columns=['Date'], inplace=True)

# Remove the first column ("Unnamed: 0")
events.drop(columns=['Unnamed: 0'], inplace=True)


In [ ]:
events

,total_sentiment,major_sent
Date,,
20080113,0.723505,positive
20080114,-0.012526,negative
20080124,0.713877,positive
20080205,-0.081316,negative
20080213,0.788888,positive
...,...,...
20171027,-0.950349,negative
20171109,-0.021314,negative
20171115,-0.867476,negative


# Regression

In [ ]:
w=factors.loc['20201001':'20201030']

In [ ]:
oneYear = timedelta(366)
oneDay = timedelta(1)

In [ ]:
date = '20171027'

In [ ]:
#In order to select indexes as expected:
#    1. Get news' date and pass it to type datetime
#    2. Subtract 1 year + 1 day to get lower limit. Then, pass it to yyyymmdd string format
#    3. Subtract 1 day to get upper limit. Then, pass it to yyyymmdd string format
#    4. Transform date to yyyymmdd to get final Rf y Ri (for the statistic)
#Loop would start here

event = datetime.strptime(date, '%Y%m%d')#.strftime('%Y%m%d')
in_low = (event-oneYear).strftime('%Y%m%d')
in_high = (event-oneDay).strftime('%Y%m%d')
event_day = event.strftime('%Y%m%d')

In [ ]:
in_low

'20161026'

In [ ]:
in_high

'20171026'

In [ ]:
allPrices.loc[in_low:in_high]['Return']

Date
20161026    0.004267
20161027   -0.001952
20161028   -0.024620
20161031   -0.017221
20161101    0.003961
              ...   
20171020    0.004472
20171023    0.001564
20171024    0.002763
20171025   -0.003594
20171026    0.003607
Name: Return, Length: 253, dtype: float64

In [ ]:
y = np.asarray(allPrices['Return'][in_low:in_high]) #Ri
x = factors.loc[in_low:in_high].to_numpy()

In [ ]:
x.shape

(253, 4)

In [ ]:
y.shape

(253,)

In [ ]:
from sklearn import linear_model

# Prepare the input and output variables
x = factors[['Constant', 'Mkt-RF', 'SMB', 'HML']].loc[in_low:in_high]
y = allPrices['Return'][in_low:in_high]

# Create and fit the OLS model
model = linear_model.LinearRegression()
model.fit(x, y)

# Extract the coefficients and intercept
a = model.intercept_
b = model.coef_[0]
s = model.coef_[1]
h = model.coef_[2]

In [ ]:
ERi=a+b*factors['Mkt-RF'][event_day]+s*factors['SMB'][event_day]+h*factors['HML'][event_day]+rf['RF'][event_day]
ERi

0.006792402599462982

In [ ]:
ERi_tot=a+b*factors['Mkt-RF'][in_low:in_high]+s*factors['SMB'][in_low:in_high]+h*factors['HML'][in_low:in_high]+rf['RF'][in_low:in_high]
Ri_tot=allPrices['Return'][in_low:in_high]

In [ ]:
abnormal_R = Ri_tot-ERi_tot.dropna()
print(abnormal_R)
day_number = Ri_tot.shape[0]

Date
20161026    0.012765
20161027    0.006759
20161028   -0.023687
20161031   -0.017538
20161101    0.007053
              ...   
20171020    0.002873
20171023    0.002118
20171024    0.002754
20171025   -0.006435
20171026    0.001157
Length: 253, dtype: float64


In [ ]:
sigma = np.sqrt(sum((Ri_tot-ERi_tot.dropna())**2)/(day_number - 2)) #Esto es el estimador
sigma_2 = np.sqrt(day_number * sigma**2)

print(sigma_2)

0.15674353700861587


In [ ]:
def get_correct_date(dicc, date):
    try:
        x = dicc[date]
        return x
    except:
        event = datetime.strptime(date, '%Y%m%d')
        return get_correct_date(dicc, (event+oneDay).strftime('%Y%m%d'))

In [ ]:
ERi_hoy=a+b*get_correct_date(factors['Mkt-RF'], date)+s*get_correct_date(factors['SMB'], date)+h*get_correct_date(factors['HML'], date)+get_correct_date(rf['RF'], date)
Ri_hoy=get_correct_date(allPrices['Return'], date)
abnormal_hoy = Ri_hoy - ERi_hoy

abnormal_ayer = abnormal_R[in_high]

In [ ]:
abnormal_ayer

0.0011573619001372182

In [ ]:
ERi_mean = ERi_tot.dropna().mean()
est=(abnormal_hoy-abnormal_ayer)/sigma

#print([Ri, sigma, est])
stats = scipy.stats.norm(0,sigma_2).cdf(est)

In [ ]:
stats

0.0005093735988299696

# Resultados

## funcion estadístico

In [ ]:
def extract_stadistics(date):
    event = datetime.strptime(date, '%Y%m%d')#.strftime('%Y%m%d')
    in_low = (event-oneYear).strftime('%Y%m%d')
    in_high = (event-oneDay).strftime('%Y%m%d')
    event_day = event.strftime('%Y%m%d')

    y = np.asarray(allPrices['Return'][in_low:in_high]) #Ri
    x = factors.loc[in_low:in_high].to_numpy()

    try:
        # Create and fit the OLS model
        model = linear_model.LinearRegression()
        model.fit(x, y)
    except:
        return None

    # Extract the coefficients and intercept
    a = model.intercept_
    b = model.coef_[0]
    s = model.coef_[1]
    h = model.coef_[2]


    ERi_tot=a+b*factors['Mkt-RF'][in_low:in_high]+s*factors['SMB'][in_low:in_high]+h*factors['HML'][in_low:in_high]+rf['RF'][in_low:in_high]
    Ri_tot=allPrices['Return'][in_low:in_high]

    abnormal_R = Ri_tot-ERi_tot.dropna()

    day_number = Ri_tot.shape[0]

    sigma = np.sqrt(sum((Ri_tot-ERi_tot.dropna())**2)/(day_number - 2)) #Esto es el estimador
    sigma_2 = np.sqrt(day_number * sigma**2)

    ERi_hoy=a+b*get_correct_date(factors['Mkt-RF'], date)+s*get_correct_date(factors['SMB'], date)+h*get_correct_date(factors['HML'], date)+get_correct_date(rf['RF'], date)
    Ri_hoy=get_correct_date(allPrices['Return'], date)
    abnormal_hoy = Ri_hoy - ERi_hoy

    abnormal_ayer = abnormal_R[-1]


    ERi_mean = ERi_tot.dropna().mean()
    est=(abnormal_hoy-abnormal_ayer)/sigma

    #print([Ri, sigma, est])
    stats = scipy.stats.norm(0,sigma_2).cdf(est)
    return stats


In [ ]:
extract_stadistics('20171027')

0.0005093735988299696

In [ ]:
events_ft_all = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/outliers_exxon_ft_all.csv')
events_ft_filter = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/outliers_exxon_ft_filtered.csv')
events_wsj_all = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/outliers_exxon_wsj_all.csv')
events_wsj_filter = pd.read_csv('/content/gdrive/MyDrive/TFG_BA/event_studies/data/outliers_exxon_wsj_filtered.csv')

## Funcion show dates

In [ ]:
def calculate_important_events(dff):
    df=dff.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    # Set the "Date" column as the index
    df.set_index(df['Date'].dt.strftime('%Y%m%d'), inplace=True)

    # Remove the "Date" column
    df.drop(columns=['Date'], inplace=True)

    # Remove the first column ("Unnamed: 0")
    df.drop(columns=['Unnamed: 0'], inplace=True)

    # Convert 'Date' to datetime if it's not already
    df.index = pd.to_datetime(df.index)

    # Filter events after 2008
    df = df[df.index.year > 2008]
    #df = df[df.index.year < 2018]

    mod_events=df

    # Now you can iterate over your filtered events and calculate statistics
    important_events = []
    for index, row in df.iterrows():
        statistics = extract_stadistics(index.strftime('%Y%m%d'))

        if statistics is not None:
            # Process the statistics or perform any desired actions
            # based on the result for each event
            if(statistics<=0.05):
                print(f"Statistics for date {index.strftime('%Y%m%d')}: {statistics}")
                important_events.append(index.strftime('%Y%m%d'))
        else:
            # Handle cases where the statistics cannot be calculated
            print(f"Statistics for date {index.strftime('%Y%m%d')} could not be calculated.")

    # Print the lengths of the filtered lists
    print("\n\n")
    print(f"Number of important events after 2008: {len(important_events)}")
    print(f"Total number of events after 2008: {len(df)}")

    # Return the list of important events and the total number of events
    return important_events, mod_events, len(important_events), len(df)



# FT

## events_ft_all

In [ ]:
important_events_ft_all, mod_events_ft_all, len_important_events, total_events = calculate_important_events(events_ft_all)

Statistics for date 20090129: 0.0033288499694684097
Statistics for date 20090227: 0.002632743972900761
Statistics for date 20090511: 0.0007559495321629666
Statistics for date 20091214: 2.866918106005522e-14
Statistics for date 20100307: 0.0019850127469640245
Statistics for date 20100311: 0.049951034791869714
Statistics for date 20100523: 1.6677794910775697e-39
Statistics for date 20100526: 2.820763623982352e-06
Statistics for date 20100603: 1.9004613678348907e-13
Statistics for date 20100609: 7.802386225655897e-138
Statistics for date 20100729: 4.570872834425412e-12
Statistics for date 20100830: 9.94293480740564e-12
Statistics for date 20100907: 0.0009143386311119958
Statistics for date 20100917: 3.869163287375544e-06
Statistics for date 20100923: 0.0019196170949997164
Statistics for date 20101022: 1.8076112252310856e-06
Statistics for date 20101102: 1.338003299800723e-06
Statistics for date 20101210: 0.01661568552377967
Statistics for date 20110105: 1.9058417800193484e-22
Statistics f

## events_ft_filter

In [ ]:
important_events_ft_filter, mod_events_ft_filter, len_important_events, total_events = calculate_important_events(events_ft_filter)

Statistics for date 20090129: 0.0033288499694684097
Statistics for date 20090227: 0.002632743972900761
Statistics for date 20090511: 0.0007559495321629666
Statistics for date 20091214: 2.866918106005522e-14
Statistics for date 20100307: 0.0019850127469640245
Statistics for date 20100523: 1.6677794910775697e-39
Statistics for date 20100526: 2.820763623982352e-06
Statistics for date 20100603: 1.9004613678348907e-13
Statistics for date 20100609: 7.802386225655897e-138
Statistics for date 20100729: 4.570872834425412e-12
Statistics for date 20100830: 9.94293480740564e-12
Statistics for date 20100907: 0.0009143386311119958
Statistics for date 20100917: 3.869163287375544e-06
Statistics for date 20100923: 0.0019196170949997164
Statistics for date 20101022: 1.8076112252310856e-06
Statistics for date 20101102: 1.338003299800723e-06
Statistics for date 20110105: 1.9058417800193484e-22
Statistics for date 20110204: 0.021262864173003715
Statistics for date 20110428: 0.00952055511992747
Statistics f

# WSJ

## events_wsj_all

In [ ]:
important_events_wsj_all, mod_events_wsj_all, len_important_events, total_events = calculate_important_events(events_wsj_all)

Statistics for date 20090103: 0.03140962336624089
Statistics for date 20090416: 0.048510983143913416
Statistics for date 20090509: 0.000737196738800258
Statistics for date 20090630: 0.04889631267652879
Statistics for date 20091202: 0.003548206676956104
Statistics for date 20100121: 0.004954383055788199
Statistics for date 20100202: 0.010857263729336009
Statistics for date 20100217: 4.0666813163772967e-10
Statistics for date 20100320: 0.021423460350012285
Statistics for date 20100330: 3.47683909011689e-09
Statistics for date 20100410: 8.415701930882398e-08
Statistics for date 20100426: 2.1322352431804288e-06
Statistics for date 20100429: 3.6120223448092744e-28
Statistics for date 20100604: 1.1948831143776625e-22
Statistics for date 20100713: 1.759241726836535e-12
Statistics for date 20100721: 1.1462905274090441e-05
Statistics for date 20100803: 4.4964142171249945e-21
Statistics for date 20101013: 0.0009081803732032839
Statistics for date 20101116: 3.4699848304420925e-05
Statistics for d

## events_wsj_filter

In [ ]:
important_events_wsj_filter, mod_events_wsj_filter, len_important_events, total_events = calculate_important_events(events_wsj_filter)

Statistics for date 20090416: 0.048510983143913416
Statistics for date 20090509: 0.000737196738800258
Statistics for date 20090630: 0.04889631267652879
Statistics for date 20091028: 0.017979027905943704
Statistics for date 20091202: 0.003548206676956104
Statistics for date 20100112: 0.0015132672473565528
Statistics for date 20100202: 0.010857263729336009
Statistics for date 20100217: 4.0666813163772967e-10
Statistics for date 20100421: 2.4800903227561334e-05
Statistics for date 20100426: 2.1322352431804288e-06
Statistics for date 20100429: 3.6120223448092744e-28
Statistics for date 20100609: 7.802386225655897e-138
Statistics for date 20100713: 1.759241726836535e-12
Statistics for date 20100721: 1.1462905274090441e-05
Statistics for date 20101013: 0.0009081803732032839
Statistics for date 20101029: 5.471957961236888e-07
Statistics for date 20101116: 3.4699848304420925e-05
Statistics for date 20101123: 0.005133581522081178
Statistics for date 20101224: 0.026491338753623
Statistics for da

# Diferencia significativa entre grupos Pos y Neg:


Prueba solo con eventos

In [ ]:
# Iterate over each row in the events dataframe
important_events = []
for index, row in events.iterrows():
    statistics = extract_stadistics(index)

    if statistics is not None:
        # Process the statistics or perform any desired actions
        # based on the result for each event
        if(statistics<=0.05):
          print(f"Statistics for date {index}: {statistics}")
          important_events.append(index)
    else:
        # Handle cases where the statistics cannot be calculated
        print(f"Statistics for date {index} could not be calculated.")


Statistics for date 20080205: 6.306519430353961e-42
Statistics for date 20080213: 0.00011699390422603637
Statistics for date 20080319: 2.8797687351573113e-108
Statistics for date 20080427: 0.021585172916189735
Statistics for date 20080501: 5.244891753551835e-49
Statistics for date 20080606: 1.5172021595925442e-68
Statistics for date 20080619: 7.024730393215072e-09
Statistics for date 20080701: 1.0774297396124781e-06
Statistics for date 20080707: 1.3474407055616672e-20
Statistics for date 20080728: 0.00010785196980911143
Statistics for date 20080731: 4.7059561128827614e-79
Statistics for date 20080817: 1.2832189557239642e-05
Statistics for date 20080915: 1.0620064329511736e-65
Statistics for date 20081007: 2.4925260126147915e-06
Statistics for date 20081009: 5.190126364682559e-33
Statistics for date 20081105: 2.1685754786015843e-09
Statistics for date 20081130: 6.046084759367008e-05
Statistics for date 20090129: 0.0033288499694684097
Statistics for date 20090311: 0.0004590057662161247
S

In [ ]:
len(important_events)

110

### funcion analyze events

In [ ]:
import pandas as pd
import scipy.stats

def analyze_events(df_events):
    # Convert the index to string format
    df_events.index = df_events.index.strftime('%Y%m%d')

    # Merge the tables based on the "Date" column
    merged_table = pd.concat([df_events, factors, allPrices, rf], axis=1, join='inner')

    # Calculate the excess return
    merged_table['Excess_Return'] = merged_table['Return'] - merged_table['RF'] / 100

    # Calculate the cumulative excess return
    merged_table['Cumulative_Excess_Return'] = merged_table.groupby('major_sent')['Excess_Return'].cumsum()

    # Calculate the average excess return for positive and negative events
    positive_events_avg = merged_table[merged_table['major_sent'] == 'positive']['Excess_Return'].mean()
    negative_events_avg = merged_table[merged_table['major_sent'] == 'negative']['Excess_Return'].mean()

    # Perform statistical tests
    t_statistic, p_value = scipy.stats.ttest_ind(
        merged_table[merged_table['major_sent'] == 'positive']['Excess_Return'],
        merged_table[merged_table['major_sent'] == 'negative']['Excess_Return']
    )

    # Print the results
    print("Average Excess Return for Positive Events:", positive_events_avg)
    print("Average Excess Return for Negative Events:", negative_events_avg)
    print("T-Statistic:", t_statistic)
    print("P-Value:", p_value)


In [ ]:
analyze_events(mod_events_ft_all)

Average Excess Return for Positive Events: 0.00022998595417323322
Average Excess Return for Negative Events: -0.00538348668355442
T-Statistic: 2.411889070766085
P-Value: 0.016980811554952242


In [ ]:
analyze_events(events)

Average Excess Return for Positive Events: 0.004377080682320948
Average Excess Return for Negative Events: -0.006023832174808543
T-Statistic: 4.213335006859334
P-Value: 3.3732768629916294e-05


H0 no hay, H1 hay diferencia significativa. Por t-tes rechazamos H0.

## Cumulative return

In [ ]:
# Merge the tables based on the "Date" column
merged_table = events.merge(factors, on='Date').merge(allPrices, on='Date').merge(rf, on='Date')

# Calculate the excess return
merged_table['Excess_Return'] = merged_table['Return'] - merged_table['RF']/100

# Calculate the cumulative excess return
merged_table['Cumulative_Excess_Return'] = merged_table.groupby('major_sent')['Excess_Return'].cumsum()

# Calculate the average excess return for positive and negative events
positive_events_avg = merged_table[merged_table['major_sent'] == 'positive']['Cumulative_Excess_Return'].mean()
negative_events_avg = merged_table[merged_table['major_sent'] == 'negative']['Cumulative_Excess_Return'].mean()

# Perform statistical tests
t_statistic, p_value = scipy.stats.ttest_ind(merged_table[merged_table['major_sent'] == 'positive']['Cumulative_Excess_Return'],
                                      merged_table[merged_table['major_sent'] == 'negative']['Cumulative_Excess_Return'])

# Print the results
print("Average Excess Return for Positive Events:", positive_events_avg)
print("Average Excess Return for Negative Events:", negative_events_avg)
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)